# Machine Learning para libros

Este proyecyo tiene la finalidad de predecir la puntuación que se ofrece a un libro según su comentario, utilizando como datos los comentarios y puntuaciones obtenidas de la tienda de Amazon en 2014.

### Creación de clases

In [1]:
import random


class Sentiment:
    NEGATIVO = 'NEGATIVO'
    NEUTRAL = 'NEUTRAL'
    POSITIVO = 'POSITIVO'

#Para que sea mas claro se crea una case con nombres:
# text, score y sentimiento
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
# Para dar un nombre a la valoración numerica, se entiende como "Sentimiento"
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVO
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #4 y 5
            return Sentiment.POSITIVO
        
# Contenedor de coemntarios

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
        # Metodo para la optención del texto
    def get_text(self):
        return [x.text for x in self.reviews]
        
        # Metodo para la optención del sentimiento
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
               
        
        # Metodo para filtrado de los sentimientos
    def evenly_distribute(self):
        positivo = list(filter(lambda x: x.sentiment == Sentiment.POSITIVO, self.reviews))
        neutral = list(filter(lambda x: x.sentiment == Sentiment.NEUTRAL, self.reviews))
        negativo = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVO, self.reviews))
        
        # Impresion para comprobación
        print('Totalidad de negativos = ', len(negativo))
        print('Totalidad de positivos = ', len(positivo))
        print('Totalidad de neutrales = ', len(neutral))
        positivos_reducido = positivo[:len(negativo)]
        self.reviews = negativo + neutral + positivos_reducido
        random.shuffle(self.reviews)
        print('Totalidad de positivos y negativos = ', len(self.reviews))
        
        
        

### Carga de Datos

In [2]:
import json

# Dirección del archivo
# Archivo con comentarios sobre libros en Amazon en 2014
# Del cual se toma en consideración el contario en texto y valoración numerica.
file_name = 'D:/Drive/MasterD/Datos varios/Coments_books.json'

# Tomar reviewText y overall y se añade a la clase Review
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
reviews[5].sentiment


'POSITIVO'

### Preparación de datos

In [3]:
# Divide en dos los datos: test 33% y train el 66%

from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42 )

# Comprobación
print(len(training))
print(len(test))



670
330


In [4]:
# Se divide texto y sentimiento en valores x y
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

Con Sklearn vectorizamos las palabras


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)
train_x_vectors

<670x7372 sparse matrix of type '<class 'numpy.int64'>'
	with 41455 stored elements in Compressed Sparse Row format>

### Clasificación


In [6]:
# Árbol de decisiones
from sklearn.tree import DecisionTreeClassifier

clasf_tree = DecisionTreeClassifier()
# Entrenar 
clasf_tree.fit(train_x_vectors, train_y)

#Se prueba con el primer comentario

print(clasf_tree.predict(test_x_vectors[0]))

# Y con el sexto

print(clasf_tree.predict(test_x_vectors[5]))

['POSITIVO']
['POSITIVO']


In [7]:
# Regresión logistica
from sklearn.linear_model import LogisticRegression

clasf_log = LogisticRegression()
# Entrenar
clasf_log.fit(train_x_vectors, train_y)

#Se prueba con el primer comentario

print(clasf_log.predict(test_x_vectors[0]))

# Y con el sexto

print(clasf_log.predict(test_x_vectors[5]))


['POSITIVO']
['NEUTRAL']


D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Evaluar los resultados


In [8]:
# Se toman los resultados de test en x(texto) comparado con y(sentimiento/valoración)
# para valorar la exactitud
print('Árbol de decisiones la exacitud es de ', clasf_tree.score(test_x_vectors, test_y), ' %')
print('Regresión logistica la exacitud es de ', clasf_log.score(test_x_vectors, test_y),' %')



Árbol de decisiones la exacitud es de  0.7757575757575758  %
Regresión logistica la exacitud es de  0.8303030303030303  %


In [9]:
# Evaluamos los resultados con la precision y el recall con F1
from sklearn.metrics import f1_score

f1_tree = f1_score(test_y, clasf_tree.predict(test_x_vectors), average = None, labels=[Sentiment.POSITIVO, Sentiment.NEUTRAL, Sentiment.NEGATIVO ])
f1_log = f1_score(test_y, clasf_log.predict(test_x_vectors), average = None, labels=[Sentiment.POSITIVO, Sentiment.NEUTRAL, Sentiment.NEGATIVO ])

print(f1_tree)
print(f1_log)

# Se puede observar que los datos se predicen correctamente en los comentarios positivos,
# Pero predicen erroneamente los cometarios neutrales y negativos.

[0.87737478 0.07142857 0.        ]
[0.91370558 0.12244898 0.1       ]


In [10]:
# Para buscar donde puede encontrarse el fallo, se estudia los datos con los que se han tomado.

# Se cuenta la cantidad de cada Sentimiento en el grupo train
print('Total Positivos ', train_y.count(Sentiment.POSITIVO))
print('Total Neutrales ', train_y.count(Sentiment.NEUTRAL))
print('Total Negativos ', train_y.count(Sentiment.NEGATIVO))

Total Positivos  552
Total Neutrales  71
Total Negativos  47


 Al estar tan desigualado la cantidad de Positivos, elentrenamiento se realiza desigualadamente, al no tener sentido usa solo 47 datos de las 3 variantes, se opta por ampliar los datos de 1.000 comentarios a 10.000 comentarios obtenidos de la misma fuente.
 
 # Inicio con nuevos datos
 
 ### Carga y Preparación de los datos

In [11]:
# Se toman los nuevos datos
file_name_2 = 'D:/Drive/MasterD/Datos varios/Coments_books_10000.json'

# Tomar reviewText y overall y se añade a la clase Review
reviews = []
with open(file_name_2) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
# Divide en dos los datos: test 33% y train el 66%
training, test = train_test_split(reviews, test_size=0.33, random_state=42 )

# Comprobación cantidad total divida en test y train
print('Totalidad de training = ', len(training))
print('Totalidad de test = ', len(test))


# Se crea dos contenedores para cantidades iguales de positivos y negativos
contenedor_train = ReviewContainer(training)
contenedor_train.evenly_distribute()

contenedor_test = ReviewContainer(test)
contenedor_test.evenly_distribute()



Totalidad de training =  6700
Totalidad de test =  3300
Totalidad de negativos =  436
Totalidad de positivos =  5611
Totalidad de neutrales =  653
Totalidad de positivos y negativos =  1525
Totalidad de negativos =  208
Totalidad de positivos =  2767
Totalidad de neutrales =  325
Totalidad de positivos y negativos =  741


In [12]:
# Se divide texto y sentimiento en valores x y, pero esta vez por metodo.
train_x = contenedor_train.get_text()
train_y = contenedor_train.get_sentiment()
test_x = contenedor_test.get_text()
test_y = contenedor_test.get_sentiment()

# Comprobación de que tanto el positivo como el negativo sea la misma cantidad de datos
print(train_y.count(Sentiment.POSITIVO))
print(train_y.count(Sentiment.NEGATIVO))


# Se vectorizan las palabras 
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

436
436


### Clasificación

In [13]:
# Árbol de decisiones
clasf_tree = DecisionTreeClassifier()

# Entrenar 
clasf_tree.fit(train_x_vectors, train_y)

#Se prueba con el primer comentario

print('Primer comentario, con árbol de decisiones ', clasf_tree.predict(test_x_vectors[0]))

# Regresión logistica
clasf_log = LogisticRegression()

# Entrenar
clasf_log.fit(train_x_vectors, train_y)

#Se prueba con el primer comentario

print('Primer comentario, con regresión Log ',clasf_log.predict(test_x_vectors[0]))



Primer comentario, con árbol de decisiones  ['POSITIVO']
Primer comentario, con regresión Log  ['NEGATIVO']


D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Se evaluan los datos

In [14]:
# Se toman los resultados de test en x(texto) comparado con y(sentimiento/valoración)
# para valorar la exactitud
print('Árbol de decisiones la exacitud es de ', clasf_tree.score(test_x_vectors, test_y), ' %')
print('Regresión logistica la exacitud es de ', clasf_log.score(test_x_vectors, test_y),' %')



Árbol de decisiones la exacitud es de  0.4534412955465587  %
Regresión logistica la exacitud es de  0.6005398110661269  %


In [15]:
# Evaluamos con F1

f1_tree = f1_score(test_y, clasf_tree.predict(test_x_vectors), average = None, labels=[Sentiment.POSITIVO, Sentiment.NEUTRAL, Sentiment.NEGATIVO ])
f1_log = f1_score(test_y, clasf_log.predict(test_x_vectors), average = None, labels=[Sentiment.POSITIVO, Sentiment.NEUTRAL, Sentiment.NEGATIVO ])

print('F1 con árbol de decisiones = ', f1_tree)
print('F1 con regresión log. = ',f1_log)

F1 con árbol de decisiones =  [0.42469136 0.50909091 0.39328537]
F1 con regresión log. =  [0.63546798 0.61102832 0.54814815]


Se observa que se ha mejorado la exactidud de las predicciones, siendo la regresión Log. la que mejor se aproxima

### Uso en ejemplos

El modelo esta entrenado y listo para su uso con ejemplos


In [16]:
test_set = ['not great', 'really bad book', 'best option', 'great time reading it']
new_test = vectorizer.transform(test_set)
clasf_log.predict(new_test)


array(['NEGATIVO', 'NEGATIVO', 'POSITIVO', 'POSITIVO'], dtype='<U8')